# Sizing Modes with Resizeable Converters

When the size of one converter is changed, it may be desirable to have other converters in the plant resized to match.
This can be done manually by setting the sizes of each converter in the  `tech_config`, but it can also be done automatically with resizeable converters.
Resizeable converters can execute their own built-in sizing methods based on how much of a feedstock can be produced upstream, or how much of a commodity can be offtaken downstream by other converters.
By connecting the capacities of converters to other converters, one can build a logical re-sizing scheme for a multi-technology plant that will resize all converters by changing just one config parameter.

## Setting up a resizeable converter

To set up a resizeable converter, take advantage of the `ResizeablePerformanceModelBaseConfig` and `ResizeablePerformanceModelBaseClass`.
The `ResizeablePerformanceModelBaseConfig` will declare a `sizing` performance parameter within the the tech_config, which is a dict that specifies the sizing mode.
The `ResizeablePerformanceModelBaseClass` will automatically parse this dict into the `inputs` and `discrete_inputs` that the performance model will need for resizing.
Here is the start of an example `tech_config` for such a converter:

In [14]:
tech_config = {
    "model_inputs": {
        "shared_parameters": {
            "production_capacity": 1000.0,
        },
        "performance_parameters": {
            "sizing": {
                "size_mode": "normal",  # Always required
                "resize_by_flow": "electricity",  # Not required in "normal" mode
                "max_feedstock_ratio": 1.6,  # Only used in "resize_by_max_feedstock"
                "max_commodity_ratio": 0.7,  # Only used in "resize_by_max_commodity"
            },
        },
    }
}

Currently, there are three different modes defined for `size_mode`:

- `normal`: In this mode, converters function as they always have previously:
    - The size of the asset is fixed within `compute()`.
- `resize_by_max_feedstock`: In this mode, the size of the converter is adjusted to be able to utilize all of the available feedstock:
    - The size of the asset should be calculated within `compute()` as a function of the maximum value of `<feedstock>_in` - with the `<feedstock>` specified by the `resize by flow` parameter.
    - This function will utilizes the `"max_feedstock_ratio"` parameter - e.g., if `"max_feedstock_ratio"` is 1.6, the converter will be resized so that its input capacity is 1.6 times the max of `<feedstock>_in`.
    - The `set_val` method will over-write any previous sizing varaibles to reflect the adjusted size of the converter.
- `resize_by_max_commodity`: In this mode, the size of the asset is adjusted to be able to supply its product to the full capacity of another downstream converter:
    - The size of the asset should be calculated within `compute()` as a function of the `max_<commodity>_capacity` input - with the `<feedstock>` specified by the `resize by flow` parameter.
    - This function will utilizes the `"max_commodity_ratio"` parameter - e.g., if `"max_commodity_ratio"` is 0.7, the converter will be resized so that its output capacity is 0.7 times a connected `"max_<commodity>_capacity"` input.
    - The `set_val` method will over-write any previous sizing varaibles to reflect the adjusted size of the converter.
    
To construct a resizeable converter from an existing converter, very few changes must be made, and only to the performance model.
`ResizeablePerformanceModelBaseConfig` can replace `BaseConfig` and `ResizeablePerformanceModelBaseClass` can replace `om.ExplicitComponent`.
The setup function must be modified to include any `"max_<feedstock>_capacity"` outputs or `"max_<commodity>_capacity"` inputs that can be connected to do the resizing. 
Then, any `feedstock_sizing_function` or `feedstock_sizing_function` that the converter needs to resize itself should be defined, if not already.
Finally, the compute method should be modified (near the start, before any size-based calculations occur) to modify the size with these functions: 

In [ ]:
import numpy as np
from h2integrate.core.utilities import ResizeablePerformanceModelBaseConfig, merge_shared_inputs
from h2integrate.core.model_baseclasses import ResizeablePerformanceModelBaseClass


class TechPerformanceModelConfig(ResizeablePerformanceModelBaseConfig):
    # Declare tech-specific config parameters
    size: float = 1.0


class TechPerformanceModel(ResizeablePerformanceModelBaseClass):
    def setup(self):
        self.config = TechPerformanceModelConfig.from_dict(
            merge_shared_inputs(self.options["tech_config"]["model_inputs"], "performance"),
            strict=False,
        )
        super().setup()

        # Declare tech-specific inputs and outputs
        self.add_input("size", val=self.config.size, units="unitless")
        # Declare any commodities produced that need to be connected to downstream converters
        # if this converter is in `resize_by_max_commodity` mode
        self.add_input("max_<commodity>_capacity", val=1000.0, units="kg/h")
        # Any feedstocks consumed that need to be connected to upstream converters
        # if those converters are in `resize_by_max_commodity` mode
        self.add_output("max_<feedstock>_capacity", val=1000.0, units="kg/h")

    def feedstock_sizing_function(max_feedstock):
        max_feedstock * 0.1231289  # random number for example

    def commodity_sizing_function(max_commodity):
        max_commodity * 0.4651  # random number for example

    def compute(self, inputs, outputs, discrete_inputs, discrete_outputs):
        size_mode = discrete_inputs["size_mode"]

        # Make changes to computation based on sizing_mode:
        if size_mode != "normal":
            size = inputs["size"]
            if size_mode == "resize_by_max_feedstock":
                if inputs["resize_by_flow"] == "<feedstock>":
                    feed_ratio = inputs["max_feedstock_ratio"]
                    size_for_max_feed = self.feedstock_sizing_function(
                        np.max(inputs["<feedstock>_in"])
                    )
                    size = size_for_max_feed * feed_ratio
            elif size_mode == "resize_by_max_commodity":
                if inputs["resize_by_flow"] == "<commodity>":
                    comm_ratio = inputs["max_commodity_ratio"]
                    size_for_max_comm = self.commodity_sizing_function(
                        np.max(inputs["max_<commodity>_capacity"])
                    )
                    size = size_for_max_comm * comm_ratio
            self.set_val("size", size)


## Example plant setup

Here, there are three technologies in the the `tech_config.yaml`:
1. A `wind` plant producing electricity,
2. An `electrolyzer` producing hydrogen from that electricity, and
3. An `ammonia` plant producting ammonia from that hydrogen.
These three technologies are all resizeable.



In [ ]:
tech_config = {
    "model_inputs": {
        "shared_parameters": {
            "production_capacity": 1000.0,
        },
        "performance_parameters": {
            "sizing": {"size_mode": "normal"},
        },
    }
}



### Example technology_interconnections within plant_config.yaml

The final connection from the ammonia back to the electroyzer is used to size the electrolyzer.

```
technology_interconnections:
- ["hopp", "electrolyzer", "electricity"]
- ["electrolyzer", "h2_storage", "hydrogen"]
- ["h2_storage", "ammonia", "hydrogen"]
- ["ammonia", "electrolyzer", "max_hydrogen_capacity"]
```

### Example tech_config for the `electrolyzer` within tech_config_size_mode_commodity_iterative.yaml

The tech_config for the electrolyzer is configured to be sized by the `resize_by_max_commodity` mode.
The sizing flow is `hydrogen`, and it will be sized to accommodate the feedstock demand of the connected technology, `ammonia`.

```
electrolyzer:
    model_inputs:
      shared_parameters:
        n_clusters: 1
        location: "onshore"
        cluster_rating_MW: 5.0
        pem_control_type: "basic"
        eol_eff_percent_loss: 10
      performance_parameters:
        sizing: 
          size_mode: "resize_by_max_commodity"
          size_flow: "hydrogen"
          size_tech: "ammonia"
```

### Example tech_config for the `ammonia` within tech_config_size_mode_commodity_iterative.yaml

For the ammonia tech_config, the sizing mode is set to `normal`, so it will be sized at the capacity value in the config.

```
ammonia:
  ...
  performance_parameters:
    sizing:
      size_mode: "normal"
    capacity_factor: 0.9
    ...
```

## Running an example

### `normal` mode

The example `.yaml` files have been set up in `normal` mode.
In this case the electrolyzer has been sized to 640 MW (same as previous example), but the electricity profile going in has a max of over 1000 MW.
The LCOH is $4.64/kg H2 and the LCOA is $1.38/kg NH3.

In [1]:
from h2integrate.core.h2integrate_model import H2IntegrateModel


# Create a H2Integrate model
model = H2IntegrateModel("14_size_mode_normal.yaml")

# Run the model
model.run()

model.post_process()

logging to stdout


78 explicit inputs:

╷                  ╷           ╷       ╷                                      
 Variable                            │             Mean │ Units     │ Shape │ Promoted name                        
╶────────────────────────────────────┼──────────────────┼───────────┼───────┼─────────────────────────────────────╴
 plant                               │                  │           │       │                                      
   ammonia                           │                  │           │       │                                      
     synloop_ammonia_cost            │                  │           │       │                                      
       total_ammonia_produced        │   328,878,670.56 │ kg/year   │ 1     │ ammonia.total_ammonia_produced       
       total_hydrogen_consumed       │    72,184,887.93 │ kg/year   │ 1     │ ammonia.total_hydrogen_consumed      
       total_nitrogen_consumed       │ 3,009,207,515.51 │ kg/year   │ 1     │ ammonia.total_nitrogen_consumed      
       total_electricity_consumed    │   245,334,157.62 │ kW*h/year │ 1     │ ammonia.total_electricity_consumed   
     synloop_ammonia_performance     │                  │           │       │                                      
       hydrogen_in                   │       8,240.2840 │ kg/h      │ 8760  │ ammonia.hydrogen_in                  
       nitrogen_in                   │           0.0000 │ kg/h      │ 8760  │ ammonia.nitrogen_in                  
       electricity_in                │           0.0000 │ MW        │ 8760  │ ammonia.electricity_in               
   electrolyzer                      │                  │           │       │                                      
     eco_pem_electrolyzer_performanc │                  │           │       │                                      
 e                                   │                  │           │       │                                      
       electricity_in                │       466,658.38 │ kW        │ 8760  │ electrolyzer.electricity_in          
     singlitico_electrolyzer_cost    │                  │           │       │                                      
       electricity_in                │       466,658.38 │ kW        │ 8760  │ electrolyzer.electricity_in          
       n_clusters                    │          16.0000 │ unitless  │ 1     │ electrolyzer.n_clusters              
       max_hydrogen_capacity         │       1,000.0000 │ kg/h      │ 1     │ electrolyzer.max_hydrogen_capacity   
       size_mode                     │              n/a │ n/a       │ n/a   │ electrolyzer.size_mode               
       total_hydrogen_produced       │    69,199,782.51 │ kg/year   │ 1     │ electrolyzer.total_hydrogen_produced 
       electrolyzer_size_mw          │         640.0000 │ MW        │ 1     │ electrolyzer.electrolyzer_size_mw    
   electrolyzer_to_h2_storage_pipe   │                  │           │       │                                      
     hydrogen_in                     │           2.2890 │ kg/s      │ 8760  │ electrolyzer_to_h2_storage_pipe.hydr 
                                     │                  │           │       │ ogen_in                              
   finance_subgroup_h2               │                  │           │       │                                      
     adjusted_capex_opex_comp        │                  │           │       │                                      
       capex_hopp                    │ 1,754,699,616.00 │ USD       │ 1     │ finance_subgroup_h2.capex_hopp       
       opex_hopp                     │    32,953,490.40 │ USD/year  │ 1     │ finance_subgroup_h2.opex_hopp        
       varopex_hopp                  │           0.0000 │ USD/year  │ 30    │ finance_subgroup_h2.varopex_hopp     
       capex_electrolyzer            │   600,412,523.71 │ USD       │ 1     │ finance_subgroup_h2.capex_electrolyz 
                                     │                  │           │       │ er             

107 explicit outputs:

╷                  ╷           ╷       ╷                                      
 Variable                            │             Mean │ Units     │ Shape │ Promoted name                        
╶────────────────────────────────────┼──────────────────┼───────────┼───────┼─────────────────────────────────────╴
 plant                               │                  │           │       │                                      
   ammonia                           │                  │           │       │                                      
     synloop_ammonia_cost            │                  │           │       │                                      
       CapEx                         │ 1,151,737,526.88 │ USD       │ 1     │ ammonia.CapEx                        
       OpEx                          │    26,006,369.17 │ USD/year  │ 1     │ ammonia.OpEx                         
       VarOpEx                       │           0.0000 │ USD/year  │ 30    │ ammonia.VarOpEx                      
       capex_asu                     │   347,075,632.70 │ USD       │ 1     │ ammonia.capex_asu                    
       capex_synloop                 │   443,088,488.84 │ USD       │ 1     │ ammonia.capex_synloop                
       capex_boiler                  │    10,355,840.22 │ USD       │ 1     │ ammonia.capex_boiler                 
       capex_cooling_tower           │     7,030,562.36 │ USD       │ 1     │ ammonia.capex_cooling_tower          
       capex_direct                  │   807,550,524.13 │ USD       │ 1     │ ammonia.capex_direct                 
       capex_depreciable_nonequipmen │   344,187,002.75 │ USD       │ 1     │ ammonia.capex_depreciable_nonequipme 
 t                                   │                  │           │       │ nt                                   
       land_cost                     │     5,015,782.62 │ USD       │ 1     │ ammonia.land_cost                    
       labor_cost                    │    28,518,441.16 │ USD/year  │ 1     │ ammonia.labor_cost                   
       general_administration_cost   │     1,445,938.00 │ USD/year  │ 1     │ ammonia.general_administration_cost  
       property_tax_insurance        │    23,034,750.54 │ USD/year  │ 1     │ ammonia.property_tax_insurance       
       maintenance_cost              │     4,037,752.62 │ USD/year  │ 1     │ ammonia.maintenance_cost             
       cost_year                     │       2,022.0000 │ n/a       │ n/a   │ ammonia.cost_year                    
     synloop_ammonia_performance     │                  │           │       │                                      
       ammonia_out                   │      37,543.2272 │ kg/h      │ 8760  │ ammonia.ammonia_out                  
       nitrogen_out                  │       308,784.88 │ kg/h      │ 8760  │ ammonia.nitrogen_out                 
       hydrogen_out                  │       1,123.8470 │ kg/h      │ 8760  │ ammonia.hydrogen_out                 
       electricity_out               │       8,084.0474 │ MW        │ 8760  │ ammonia.electricity_out              
       heat_out                      │      31,160.7134 │ kW*h/kg   │ 8760  │ ammonia.heat_out                     
       catalyst_mass                 │      90,075.2840 │ kg        │ 1     │ ammonia.catalyst_mass                
       total_ammonia_produced        │   328,878,670.56 │ kg/year   │ 1     │ ammonia.total_ammonia_produced       
       total_hydrogen_consumed       │    72,184,887.93 │ kg/year   │ 1     │ ammonia.total_hydrogen_consumed      
       total_nitrogen_consumed       │ 3,009,207,515.51 │ kg/year   │ 1     │ ammonia.total_nitrogen_consumed      
       total_electricity_consumed    │   245,334,157.62 │ kW*h/year │ 1     │ ammonia.total_electricity_consumed   
       limiting_input                │           1.8242 │ n/a       │ 8760  │ ammonia.limiting_input               
       max_hydrogen_capacity         │      10,589.3601 │ kg/h      │ 1     │ ammonia.max_hyd

### `resize_by_max_feedstock` mode

In this case, the electrolyzer will be sized to match the maximum `electricity_in` coming from HOPP
This increases the electrolyzer size to 1080 MW, the closest multiple of 40 MW (the cluster size) matching the max HOPP power output of 1048 MW.
This increases the LCOH to $4.98/kg H2, and increases the LCOA to $1.58/kg NH3, since electrolyzer is now oversized to utilize all of the HOPP electricity at peak output but thus has a lower hydrogen production capacity factor.

In [2]:
# Create a H2Integrate model
feed_model = H2IntegrateModel("14_size_mode_feedstock.yaml")

# Run the model
feed_model.run()

feed_model.post_process()

80 explicit inputs:

╷                  ╷           ╷       ╷                                      
 Variable                            │             Mean │ Units     │ Shape │ Promoted name                        
╶────────────────────────────────────┼──────────────────┼───────────┼───────┼─────────────────────────────────────╴
 plant                               │                  │           │       │                                      
   ammonia                           │                  │           │       │                                      
     synloop_ammonia_cost            │                  │           │       │                                      
       total_ammonia_produced        │   332,015,828.22 │ kg/year   │ 1     │ ammonia.total_ammonia_produced       
       total_hydrogen_consumed       │    82,091,535.03 │ kg/year   │ 1     │ ammonia.total_hydrogen_consumed      
       total_nitrogen_consumed       │ 3,422,190,866.59 │ kg/year   │ 1     │ ammonia.total_nitrogen_consumed      
       total_electricity_consumed    │   245,334,157.62 │ kW*h/year │ 1     │ ammonia.total_electricity_consumed   
     synloop_ammonia_performance     │                  │           │       │                                      
       hydrogen_in                   │       9,371.1798 │ kg/h      │ 8760  │ ammonia.hydrogen_in                  
       nitrogen_in                   │           0.0000 │ kg/h      │ 8760  │ ammonia.nitrogen_in                  
       electricity_in                │           0.0000 │ MW        │ 8760  │ ammonia.electricity_in               
   electrolyzer                      │                  │           │       │                                      
     eco_pem_electrolyzer_performanc │                  │           │       │                                      
 e                                   │                  │           │       │                                      
       max_feedstock_ratio           │           1.0000 │ unitless  │ 1     │ electrolyzer.max_feedstock_ratio     
       electricity_in                │       466,658.38 │ kW        │ 8760  │ electrolyzer.electricity_in          
     singlitico_electrolyzer_cost    │                  │           │       │                                      
       electricity_in                │       466,658.38 │ kW        │ 8760  │ electrolyzer.electricity_in          
       n_clusters                    │          16.0000 │ unitless  │ 1     │ electrolyzer.n_clusters              
       max_hydrogen_capacity         │       1,000.0000 │ kg/h      │ 1     │ electrolyzer.max_hydrogen_capacity   
       size_mode                     │              n/a │ n/a       │ n/a   │ electrolyzer.size_mode               
       resize_by_flow                │              n/a │ n/a       │ n/a   │ electrolyzer.resize_by_flow          
       total_hydrogen_produced       │    78,752,543.84 │ kg/year   │ 1     │ electrolyzer.total_hydrogen_produced 
       electrolyzer_size_mw          │       1,080.0000 │ MW        │ 1     │ electrolyzer.electrolyzer_size_mw    
   electrolyzer_to_h2_storage_pipe   │                  │           │       │                                      
     hydrogen_in                     │           2.6031 │ kg/s      │ 8760  │ electrolyzer_to_h2_storage_pipe.hydr 
                                     │                  │           │       │ ogen_in                              
   finance_subgroup_h2               │                  │           │       │                                      
     adjusted_capex_opex_comp        │                  │           │       │                                      
       capex_hopp                    │ 1,754,699,616.00 │ USD       │ 1     │ finance_subgroup_h2.capex_hopp       
       opex_hopp                     │    32,953,490.40 │ USD/year  │ 1     │ finance_subgroup_h2.opex_hopp        
       varopex_hopp                  │           0.0000 │ USD/year  │ 30    │ finance_subgrou

107 explicit outputs:

╷                  ╷           ╷       ╷                                      
 Variable                            │             Mean │ Units     │ Shape │ Promoted name                        
╶────────────────────────────────────┼──────────────────┼───────────┼───────┼─────────────────────────────────────╴
 plant                               │                  │           │       │                                      
   ammonia                           │                  │           │       │                                      
     synloop_ammonia_cost            │                  │           │       │                                      
       CapEx                         │ 1,151,737,526.88 │ USD       │ 1     │ ammonia.CapEx                        
       OpEx                          │    25,982,406.63 │ USD/year  │ 1     │ ammonia.OpEx                         
       VarOpEx                       │           0.0000 │ USD/year  │ 30    │ ammonia.VarOpEx                      
       capex_asu                     │   347,075,632.70 │ USD       │ 1     │ ammonia.capex_asu                    
       capex_synloop                 │   443,088,488.84 │ USD       │ 1     │ ammonia.capex_synloop                
       capex_boiler                  │    10,355,840.22 │ USD       │ 1     │ ammonia.capex_boiler                 
       capex_cooling_tower           │     7,030,562.36 │ USD       │ 1     │ ammonia.capex_cooling_tower          
       capex_direct                  │   807,550,524.13 │ USD       │ 1     │ ammonia.capex_direct                 
       capex_depreciable_nonequipmen │   344,187,002.75 │ USD       │ 1     │ ammonia.capex_depreciable_nonequipme 
 t                                   │                  │           │       │ nt                                   
       land_cost                     │     5,015,782.62 │ USD       │ 1     │ ammonia.land_cost                    
       labor_cost                    │    28,518,441.16 │ USD/year  │ 1     │ ammonia.labor_cost                   
       general_administration_cost   │     1,445,938.00 │ USD/year  │ 1     │ ammonia.general_administration_cost  
       property_tax_insurance        │    23,034,750.54 │ USD/year  │ 1     │ ammonia.property_tax_insurance       
       maintenance_cost              │     4,037,752.62 │ USD/year  │ 1     │ ammonia.maintenance_cost             
       cost_year                     │       2,022.0000 │ n/a       │ n/a   │ ammonia.cost_year                    
     synloop_ammonia_performance     │                  │           │       │                                      
       ammonia_out                   │      37,901.3503 │ kg/h      │ 8760  │ ammonia.ammonia_out                  
       nitrogen_out                  │       355,597.79 │ kg/h      │ 8760  │ ammonia.nitrogen_out                 
       hydrogen_out                  │       2,186.8594 │ kg/h      │ 8760  │ ammonia.hydrogen_out                 
       electricity_out               │       7,894.0112 │ MW        │ 8760  │ ammonia.electricity_out              
       heat_out                      │      31,457.9539 │ kW*h/kg   │ 8760  │ ammonia.heat_out                     
       catalyst_mass                 │      90,934.5078 │ kg        │ 1     │ ammonia.catalyst_mass                
       total_ammonia_produced        │   332,015,828.22 │ kg/year   │ 1     │ ammonia.total_ammonia_produced       
       total_hydrogen_consumed       │    82,091,535.03 │ kg/year   │ 1     │ ammonia.total_hydrogen_consumed      
       total_nitrogen_consumed       │ 3,422,190,866.59 │ kg/year   │ 1     │ ammonia.total_nitrogen_consumed      
       total_electricity_consumed    │   245,334,157.62 │ kW*h/year │ 1     │ ammonia.total_electricity_consumed   
       limiting_input                │           1.8502 │ n/a       │ 8760  │ ammonia.limiting_input               
       max_hydrogen_capacity         │      10,589.3601 │ kg/h      │ 1     │ ammonia.max_hyd

### `resize_by_max_product` mode

In this case, the electrolyzer will be sized to match the maximum `hydrogen_in` used by ammonia production
This decreases the electrolyzer size to 560 MW, the closest multiple of 40 MW (the cluster size) that will ensure an h2 produciton capacity that matches the ammonia plant's h2 intake at its max ammonia produciton capacity.
This increases the LCOH to $4.78/kg H2, but reduces the LCOA to $1.33/kg NH3, since electrolyzer size was matched to ammonia produciton but not HOPP.

In [3]:
# Create a H2Integrate model
prod_model = H2IntegrateModel("14_size_mode_commodity.yaml")

# Run the model
prod_model.run()

prod_model.post_process()


=====
plant
=====
NL: NLBGS Converged in 5 iterations


81 explicit inputs:

╷                  ╷           ╷       ╷                                      
 Variable                            │             Mean │ Units     │ Shape │ Promoted name                        
╶────────────────────────────────────┼──────────────────┼───────────┼───────┼─────────────────────────────────────╴
 plant                               │                  │           │       │                                      
   ammonia                           │                  │           │       │                                      
     synloop_ammonia_cost            │                  │           │       │                                      
       total_ammonia_produced        │   327,187,769.66 │ kg/year   │ 1     │ ammonia.total_ammonia_produced       
       total_hydrogen_consumed       │    65,763,750.11 │ kg/year   │ 1     │ ammonia.total_hydrogen_consumed      
       total_nitrogen_consumed       │ 2,741,526,332.57 │ kg/year   │ 1     │ ammonia.total_nitrogen_consumed      
       total_electricity_consumed    │   245,334,157.62 │ kW*h/year │ 1     │ ammonia.total_electricity_consumed   
     synloop_ammonia_performance     │                  │           │       │                                      
       hydrogen_in                   │       7,507.2774 │ kg/h      │ 8760  │ ammonia.hydrogen_in                  
       nitrogen_in                   │           0.0000 │ kg/h      │ 8760  │ ammonia.nitrogen_in                  
       electricity_in                │           0.0000 │ MW        │ 8760  │ ammonia.electricity_in               
   electrolyzer                      │                  │           │       │                                      
     eco_pem_electrolyzer_performanc │                  │           │       │                                      
 e                                   │                  │           │       │                                      
       max_commodity_ratio           │           1.0000 │ unitless  │ 1     │ electrolyzer.max_commodity_ratio     
       electricity_in                │       466,658.38 │ kW        │ 8760  │ electrolyzer.electricity_in          
     singlitico_electrolyzer_cost    │                  │           │       │                                      
       electricity_in                │       466,658.38 │ kW        │ 8760  │ electrolyzer.electricity_in          
       n_clusters                    │          16.0000 │ unitless  │ 1     │ electrolyzer.n_clusters              
       max_hydrogen_capacity         │      10,589.3601 │ kg/h      │ 1     │ electrolyzer.max_hydrogen_capacity   
       size_mode                     │              n/a │ n/a       │ n/a   │ electrolyzer.size_mode               
       resize_by_flow                │              n/a │ n/a       │ n/a   │ electrolyzer.resize_by_flow          
       resize_by_tech                │              n/a │ n/a       │ n/a   │ electrolyzer.resize_by_tech          
       total_hydrogen_produced       │    63,047,007.97 │ kg/year   │ 1     │ electrolyzer.total_hydrogen_produced 
       electrolyzer_size_mw          │         560.0000 │ MW        │ 1     │ electrolyzer.electrolyzer_size_mw    
   electrolyzer_to_h2_storage_pipe   │                  │           │       │                                      
     hydrogen_in                     │           2.0854 │ kg/s      │ 8760  │ electrolyzer_to_h2_storage_pipe.hydr 
                                     │                  │           │       │ ogen_in                              
   finance_subgroup_h2               │                  │           │       │                                      
     adjusted_capex_opex_comp        │                  │           │       │                                      
       capex_hopp                    │ 1,754,699,616.00 │ USD       │ 1     │ finance_subgroup_h2.capex_hopp       
       opex_hopp                     │    32,953,490.40 │ USD/year  │ 1     │ finance_subgrou

107 explicit outputs:

╷                  ╷           ╷       ╷                                      
 Variable                            │             Mean │ Units     │ Shape │ Promoted name                        
╶────────────────────────────────────┼──────────────────┼───────────┼───────┼─────────────────────────────────────╴
 plant                               │                  │           │       │                                      
   ammonia                           │                  │           │       │                                      
     synloop_ammonia_cost            │                  │           │       │                                      
       CapEx                         │ 1,151,737,526.88 │ USD       │ 1     │ ammonia.CapEx                        
       OpEx                          │    26,019,284.76 │ USD/year  │ 1     │ ammonia.OpEx                         
       VarOpEx                       │           0.0000 │ USD/year  │ 30    │ ammonia.VarOpEx                      
       capex_asu                     │   347,075,632.70 │ USD       │ 1     │ ammonia.capex_asu                    
       capex_synloop                 │   443,088,488.84 │ USD       │ 1     │ ammonia.capex_synloop                
       capex_boiler                  │    10,355,840.22 │ USD       │ 1     │ ammonia.capex_boiler                 
       capex_cooling_tower           │     7,030,562.36 │ USD       │ 1     │ ammonia.capex_cooling_tower          
       capex_direct                  │   807,550,524.13 │ USD       │ 1     │ ammonia.capex_direct                 
       capex_depreciable_nonequipmen │   344,187,002.75 │ USD       │ 1     │ ammonia.capex_depreciable_nonequipme 
 t                                   │                  │           │       │ nt                                   
       land_cost                     │     5,015,782.62 │ USD       │ 1     │ ammonia.land_cost                    
       labor_cost                    │    28,518,441.16 │ USD/year  │ 1     │ ammonia.labor_cost                   
       general_administration_cost   │     1,445,938.00 │ USD/year  │ 1     │ ammonia.general_administration_cost  
       property_tax_insurance        │    23,034,750.54 │ USD/year  │ 1     │ ammonia.property_tax_insurance       
       maintenance_cost              │     4,037,752.62 │ USD/year  │ 1     │ ammonia.maintenance_cost             
       cost_year                     │       2,022.0000 │ n/a       │ n/a   │ ammonia.cost_year                    
     synloop_ammonia_performance     │                  │           │       │                                      
       ammonia_out                   │      37,350.2020 │ kg/h      │ 8760  │ ammonia.ammonia_out                  
       nitrogen_out                  │       278,406.24 │ kg/h      │ 8760  │ ammonia.nitrogen_out                 
       hydrogen_out                  │         427.4289 │ kg/h      │ 8760  │ ammonia.hydrogen_out                 
       electricity_out               │       8,186.4754 │ MW        │ 8760  │ ammonia.electricity_out              
       heat_out                      │      31,000.5033 │ kW*h/kg   │ 8760  │ ammonia.heat_out                     
       catalyst_mass                 │      89,612.1698 │ kg        │ 1     │ ammonia.catalyst_mass                
       total_ammonia_produced        │   327,187,769.66 │ kg/year   │ 1     │ ammonia.total_ammonia_produced       
       total_hydrogen_consumed       │    65,763,750.11 │ kg/year   │ 1     │ ammonia.total_hydrogen_consumed      
       total_nitrogen_consumed       │ 2,741,526,332.57 │ kg/year   │ 1     │ ammonia.total_nitrogen_consumed      
       total_electricity_consumed    │   245,334,157.62 │ kW*h/year │ 1     │ ammonia.total_electricity_consumed   
       limiting_input                │           1.4868 │ n/a       │ 8760  │ ammonia.limiting_input               
       max_hydrogen_capacity         │      10,589.3601 │ kg/h      │ 1     │ ammonia.max_hyd

## Using optimizer with `max_feedstock_ratio` and `max_commodity_ratio`

In `size_by_max_feedstock` mode, the optimizer finds optimal `max_feedstock_ratio` of 0.515 to minimize LCOA with fixed HOPP & ammonia plant sizes.

In [9]:
# Create a H2Integrate model
prod_model = H2IntegrateModel("14_size_mode_optimizer.yaml")

# Run the model
prod_model.run()

prod_model.post_process()

Driver debug print for iter coord: rank0:ScipyOptimize_COBYLA|0
---------------------------------------------------------------
Design Vars
{'electrolyzer.max_feedstock_ratio': array([1.])}

Nonlinear constraints
None

Linear constraints
None

Objectives
{'finance_subgroup_nh3.LCOA': array([1.58471263])}

Driver debug print for iter coord: rank0:ScipyOptimize_COBYLA|1
---------------------------------------------------------------
Design Vars
{'electrolyzer.max_feedstock_ratio': array([1.])}

Nonlinear constraints
None

Linear constraints
None

Objectives
{'finance_subgroup_nh3.LCOA': array([1.58471263])}

Driver debug print for iter coord: rank0:ScipyOptimize_COBYLA|2
---------------------------------------------------------------
Design Vars
{'electrolyzer.max_feedstock_ratio': array([1.1])}

Nonlinear constraints
None

Linear constraints
None

Objectives
{'finance_subgroup_nh3.LCOA': array([1.61935128])}

Driver debug print for iter coord: rank0:ScipyOptimize_COBYLA|3
--------------

80 explicit inputs:

╷                  ╷           ╷       ╷                                      
 Variable                            │             Mean │ Units     │ Shape │ Promoted name                        
╶────────────────────────────────────┼──────────────────┼───────────┼───────┼─────────────────────────────────────╴
 plant                               │                  │           │       │                                      
   ammonia                           │                  │           │       │                                      
     synloop_ammonia_cost            │                  │           │       │                                      
       total_ammonia_produced        │   327,187,769.66 │ kg/year   │ 1     │ ammonia.total_ammonia_produced       
       total_hydrogen_consumed       │    65,763,750.11 │ kg/year   │ 1     │ ammonia.total_hydrogen_consumed      
       total_nitrogen_consumed       │ 2,741,526,332.57 │ kg/year   │ 1     │ ammonia.total_nitrogen_consumed      
       total_electricity_consumed    │   245,334,157.62 │ kW*h/year │ 1     │ ammonia.total_electricity_consumed   
     synloop_ammonia_performance     │                  │           │       │                                      
       hydrogen_in                   │       7,507.2774 │ kg/h      │ 8760  │ ammonia.hydrogen_in                  
       nitrogen_in                   │           0.0000 │ kg/h      │ 8760  │ ammonia.nitrogen_in                  
       electricity_in                │           0.0000 │ MW        │ 8760  │ ammonia.electricity_in               
   electrolyzer                      │                  │           │       │                                      
     eco_pem_electrolyzer_performanc │                  │           │       │                                      
 e                                   │                  │           │       │                                      
       max_feedstock_ratio           │           0.5150 │ unitless  │ 1     │ electrolyzer.max_feedstock_ratio     
       electricity_in                │       466,658.38 │ kW        │ 8760  │ electrolyzer.electricity_in          
     singlitico_electrolyzer_cost    │                  │           │       │                                      
       electricity_in                │       466,658.38 │ kW        │ 8760  │ electrolyzer.electricity_in          
       n_clusters                    │          16.0000 │ unitless  │ 1     │ electrolyzer.n_clusters              
       max_hydrogen_capacity         │       1,000.0000 │ kg/h      │ 1     │ electrolyzer.max_hydrogen_capacity   
       size_mode                     │              n/a │ n/a       │ n/a   │ electrolyzer.size_mode               
       resize_by_flow                │              n/a │ n/a       │ n/a   │ electrolyzer.resize_by_flow          
       total_hydrogen_produced       │    63,047,007.97 │ kg/year   │ 1     │ electrolyzer.total_hydrogen_produced 
       electrolyzer_size_mw          │         560.0000 │ MW        │ 1     │ electrolyzer.electrolyzer_size_mw    
   electrolyzer_to_h2_storage_pipe   │                  │           │       │                                      
     hydrogen_in                     │           2.0854 │ kg/s      │ 8760  │ electrolyzer_to_h2_storage_pipe.hydr 
                                     │                  │           │       │ ogen_in                              
   finance_subgroup_h2               │                  │           │       │                                      
     adjusted_capex_opex_comp        │                  │           │       │                                      
       capex_hopp                    │ 1,754,699,616.00 │ USD       │ 1     │ finance_subgroup_h2.capex_hopp       
       opex_hopp                     │    32,953,490.40 │ USD/year  │ 1     │ finance_subgroup_h2.opex_hopp        
       varopex_hopp                  │           0.0000 │ USD/year  │ 30    │ finance_subgrou

107 explicit outputs:

╷                  ╷           ╷       ╷                                      
 Variable                            │             Mean │ Units     │ Shape │ Promoted name                        
╶────────────────────────────────────┼──────────────────┼───────────┼───────┼─────────────────────────────────────╴
 plant                               │                  │           │       │                                      
   ammonia                           │                  │           │       │                                      
     synloop_ammonia_cost            │                  │           │       │                                      
       CapEx                         │ 1,151,737,526.88 │ USD       │ 1     │ ammonia.CapEx                        
       OpEx                          │    26,019,284.76 │ USD/year  │ 1     │ ammonia.OpEx                         
       VarOpEx                       │           0.0000 │ USD/year  │ 30    │ ammonia.VarOpEx                      
       capex_asu                     │   347,075,632.70 │ USD       │ 1     │ ammonia.capex_asu                    
       capex_synloop                 │   443,088,488.84 │ USD       │ 1     │ ammonia.capex_synloop                
       capex_boiler                  │    10,355,840.22 │ USD       │ 1     │ ammonia.capex_boiler                 
       capex_cooling_tower           │     7,030,562.36 │ USD       │ 1     │ ammonia.capex_cooling_tower          
       capex_direct                  │   807,550,524.13 │ USD       │ 1     │ ammonia.capex_direct                 
       capex_depreciable_nonequipmen │   344,187,002.75 │ USD       │ 1     │ ammonia.capex_depreciable_nonequipme 
 t                                   │                  │           │       │ nt                                   
       land_cost                     │     5,015,782.62 │ USD       │ 1     │ ammonia.land_cost                    
       labor_cost                    │    28,518,441.16 │ USD/year  │ 1     │ ammonia.labor_cost                   
       general_administration_cost   │     1,445,938.00 │ USD/year  │ 1     │ ammonia.general_administration_cost  
       property_tax_insurance        │    23,034,750.54 │ USD/year  │ 1     │ ammonia.property_tax_insurance       
       maintenance_cost              │     4,037,752.62 │ USD/year  │ 1     │ ammonia.maintenance_cost             
       cost_year                     │       2,022.0000 │ n/a       │ n/a   │ ammonia.cost_year                    
     synloop_ammonia_performance     │                  │           │       │                                      
       ammonia_out                   │      37,350.2020 │ kg/h      │ 8760  │ ammonia.ammonia_out                  
       nitrogen_out                  │       278,406.24 │ kg/h      │ 8760  │ ammonia.nitrogen_out                 
       hydrogen_out                  │         427.4289 │ kg/h      │ 8760  │ ammonia.hydrogen_out                 
       electricity_out               │       8,186.4754 │ MW        │ 8760  │ ammonia.electricity_out              
       heat_out                      │      31,000.5033 │ kW*h/kg   │ 8760  │ ammonia.heat_out                     
       catalyst_mass                 │      89,612.1698 │ kg        │ 1     │ ammonia.catalyst_mass                
       total_ammonia_produced        │   327,187,769.66 │ kg/year   │ 1     │ ammonia.total_ammonia_produced       
       total_hydrogen_consumed       │    65,763,750.11 │ kg/year   │ 1     │ ammonia.total_hydrogen_consumed      
       total_nitrogen_consumed       │ 2,741,526,332.57 │ kg/year   │ 1     │ ammonia.total_nitrogen_consumed      
       total_electricity_consumed    │   245,334,157.62 │ kW*h/year │ 1     │ ammonia.total_electricity_consumed   
       limiting_input                │           1.4868 │ n/a       │ 8760  │ ammonia.limiting_input               
       max_hydrogen_capacity         │      10,589.3601 │ kg/h      │ 1     │ ammonia.max_hyd